<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #3: Upload clips to Zooniverse</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Apr 20, 2022</h5>

# Set up and requirements

### Import Python packages

In [ ]:
%load_ext autoreload
%autoreload 2

# Set the directory of the libraries
import sys
sys.path.append('..')

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.t3_utils as t3
import kso_utils.project_utils as p_utils
import kso_utils.server_utils as s_utils

print("Packages loaded successfully")

### Choose your project

In [ ]:
project_name = t_utils.choose_project()

In [ ]:
project = p_utils.find_project(project_name=project_name.value)

### Initiate sql and zoo project

In [ ]:
# Initiate db
db_info_dict = t_utils.initiate_db(project)

In [ ]:
# Connect to Zooniverse project
zoo_project = t_utils.connect_zoo_project(project)

### Retrieve info about zooniverse clips

In [ ]:
zoo_info_dict = t_utils.retrieve__populate_zoo_info(
    project = project, 
    db_info_dict = db_info_dict,
    zoo_project = zoo_project,
    zoo_info = ["subjects"]
)

### Retrieve info about movies hosted in the server

In [ ]:
# Store info about the movies available in the server
available_movies_df = s_utils.retrieve_movie_info_from_server(
    project = project,
    db_info_dict = db_info_dict
)

# Select the movie of interest

### Select the movie you want to upload to Zooniverse

In [ ]:
movie_selected = t3.select_movie(available_movies_df)

### Preview the movie selected

In [ ]:
movie_display, movie_path = t_utils.preview_movie(
    project = project,
    db_info_dict = db_info_dict, 
    available_movies_df = available_movies_df, 
    movie_i = movie_selected.value
)
movie_display

### Check if movie is already in Zooniverse

Remember to query the newest zooniverse data to get the most up to date list of clips uploaded

In [ ]:
t3.check_movie_uploaded(
    movie_i = movie_selected.value,
    db_info_dict = db_info_dict
)

# Create some clip examples

### Create a few clip examples

Test different parameters (e.g. compression rate, color modifications) in randomly selected clip examples

In [ ]:
random_clip_selection = t3.select_random_clips(
    movie_i = movie_selected.value,
    db_info_dict = db_info_dict
)

In [ ]:
example_clips = t3.create_example_clips(
    movie_i = movie_selected.value,
    movie_path = movie_path,
    db_info_dict = db_info_dict,
    project = project,
    clip_selection = random_clip_selection
)

### Review the size of the clip examples

In [ ]:
t3.check_clip_size(clips_list = example_clips)

### Select the clip modification

In [ ]:
clip_modification = t3.clip_modification_widget()
clip_modification

### Specify the GPU availability

In [ ]:
gpu_available = t3.gpu_select()

### Modify the clip examples

In [ ]:
modified_clips = t3.create_modified_clips(
    clips_list = example_clips,
    movie_i = movie_selected.value,
    modification_details = clip_modification.checks,
    project = project,
    gpu_available = gpu_available.result
)

### Review the size of the modified clips

In [ ]:
t3.check_clip_size(clips_list = modified_clips)

### Compare the clips

In [ ]:
t3.compare_clips(
    random_clips = random_clips,
    modified_clips = modified_clips
)

# Create the clips to upload to Zooniverse

### Preview the movie selected

In [ ]:
movie_display

### Specify the number of clips and clip length

In [ ]:
clip_selection = t3.select_clip_n_len(
    movie_i = movie_selected.value,
    db_info_dict = db_info_dict
)

### Review the clips that will be created

In [ ]:
t3.review_clip_selection(
    clip_selection = clip_selection, 
    movie_i = movie_selected.value,
    clip_modification = clip_modification
)

### Create the clips

In [ ]:
clips_to_upload_df = t3.create_clips(
    available_movies_df = available_movies_df, 
    movie_i = movie_selected.value,
    movie_path = movie_path,
    db_info_dict = db_info_dict,
    clip_selection = clip_selection, 
    project = project, 
    modification_details = clip_modification.checks,
    gpu_available = gpu_available.result
)

### Review the size of the clips

In [ ]:
t3.check_clip_size(clips_df = clips_to_upload_df)

### Set Zooniverse metadata

Make sure your workflows in Zooniverse have different names to avoid issues while selecting the workflow id

In [ ]:
upload_to_zoo, sitename, created_on = t3.set_zoo_metadata(
    db_info_dict = db_info_dict, 
    df = clips_to_upload_df,
    project = project
    )

### Upload clips to Zooniverse

You may receive an error message related to file size if clips exceed the recommended limit for Zooniverse uploads. In this case, we recommend shortening the clip length to achieve a suitable filesize.

In [ ]:
t3.upload_clips_to_zooniverse(
    upload_to_zoo = upload_to_zoo, 
    sitename = sitename,
    created_on = created_on,
    project = project.Zooniverse_number
)

In [ ]:
#END